# 추천 시스템

 - 협업 필터링 : 다른 사용자의 과거 선택 이력을 기반으로 추천해주는 시스템
 
 - 콘텐츠 기반 필터링 : 콘텐츠를 분석해서 그 사람이 좋아할 만한 콘텐츠를 추천.


- 협업 필터링의 단점 
1. 충분한 정보가 없거나 새로운 정보에 대해서는 추천이 안돼
2. 사용자가 많으면 계산량이 많아져 효율이 떨어짐
3. 사용자들이 사용 안한것들은 추천하지 못해

### 추천 시스템  
### 1. 아이템이 매우 많고
### 2. 유저의 취향이 다양할 때  예측을 잘한다.

```
1) 작업디렉토리 생성
$ mkdir -p ~/aiffel/recommendata_iu/data

2) wget으로 데이터 다운로드 (주의) 오래걸립니다. 
$ wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz

3) 다운받은 데이터를 작업디렉토리로 이동
$ mv lastfm-dataset-360K.tar.gz ~/aiffel/recommendata_iu/data

4) gzip으로 압축된 압축을 해제하면 tar 파일이 하나 나옵니다. 
$ gunzip lastfm-dataset-360K.tar.gz

5) tar 압축을 다시 해제하면 우리가 사용할 최종 데이터 파일이 나옵니다. 
$ tar -xvf lastfm-dataset-360K.tar

6) 필요 라이브러리 설치
$ pip install implicit
```

 - 메모리 상에서 데이터 열어서 확인해보자.
```
$ more ~/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv
```

데이터 구조 : http://ocelma.net/MusicRecommendationDataset/lastfm-360K.html

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [3]:
# artist -MBID는 필요 없다.사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)


,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
data['artist'] = data['artist'].str.lower() 
# 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [5]:
condition = data['user_id']== data.loc[0, 'user_id']
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [6]:
# 유저 수
data['user_id'].nunique()

358868

In [7]:
# 아티스트 수
data['artist'].nunique()

291346

In [8]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [9]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [10]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

### 모델 검증을 위한 사용자 초기 정보 세팅하기
 - 유튜브 추천 시스템 처럼, 5명의 아티스트 선택하면 좋아하는 아티스트들을 추천해주려고 함.

In [11]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


In [12]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [35]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing Fail!!
artist column indexing Fail!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


### 사용자의 explicit, Implicit Dataset.
https://orill.tistory.com/entry/Explicit-vs-Implicit-Feedback-Datasets?category=1066301

In [15]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  
# f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


## 가정
### 한 번이라도 들었으면 선호한다고 판단한다.
### 많이 재생한 아티스트에 대해 가중치를 주어 더 좋아한다고 판단한다.

## Matrix Factorization(MF) model

 - m명의 사용자들이 n명의 아티스트에 대해 평가한 데이터 평가행렬(Rating Matrix)를 만들거다.
 - 협업 필터링은 결국 평가행렬을 전제로 하는 것.
 
 - MF 모델의 기본 아이디어 : (m,n) 사이즈의 행렬을 R(m,k) 와 Q(k,n)행렬로 분해하여 표현함. K는 작은 값이어서 계산량이 매우 적다.
 
 ![img](im1.png)
 
 
 Feature는 K차원의 백터를 사용자 수 만큼 모아놓은 행렬.
 
 P0(1,0.1) 은 빨간 모자를 쓴 첫번째 사용자 특성 벡터
 Q0(0.9,-0.2)는 해리포터 영화의 특성 벡터.
 R00(0.88)이 사용자의 영화 선호도.
 
  ![img](im2.png)
 
 [출처: https://developers.google.com/machine-learning/recommendation/collaborative/matrix]

### CSR (Compressed Sparse Row) Matrix

 - 데이터가 너무 클 때, 줄이기 위한 방법.
 
 36만 * 29만 * 1byte =97GB . 너무 크다.!
 
 CSR matrix를 사용해 데이터 크기를 줄인다.
 https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005

```
data = np.array([1,3,5,6,2,11,7,9,10,12])
   indices = np.array([1,2,3,0,1,3,1,2,2,3])
   indptr  = np.array([0,3,6,8,10])
```

data = 데이터 값.

indices = column index 값.

indptr = row 당 들어있는 값의 갯수

   

In [16]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

### MF 모델 학습하기

 - implicit 패키지로 학습심키기.
 - 이 패키지에 구현된 als(AlternatingLeastSquares)모델을 사용

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

In [19]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]


In [20]:
zimin_vector

array([-0.1774159 ,  1.0307624 ,  0.4287772 , -0.11229997,  0.5388273 ,
        0.717291  ,  0.24737023,  0.56550974,  0.23188269, -0.2995214 ,
       -0.61408615,  0.04184856,  1.4365975 , -0.580557  ,  0.6803088 ,
       -0.393198  , -1.4165831 , -0.6455069 , -0.91907334, -0.04162171,
       -0.30867723, -0.2852374 ,  0.11895189,  0.9886565 , -0.66046196,
       -0.26683903,  0.22083716,  0.5429477 ,  0.20913574,  0.10502801,
        0.9599674 ,  0.6142643 ,  0.3194834 , -0.51870334, -0.57719797,
       -0.45506886,  0.7784327 ,  1.2856175 , -0.30071786, -0.34609607,
       -1.2056104 , -0.0560937 ,  0.58504945,  1.7960583 , -0.1360319 ,
        0.12275474,  0.07696783, -0.32553887, -0.6765309 , -0.9411583 ,
        0.053318  , -0.89167476, -0.09722766,  1.083515  ,  0.3834413 ,
       -1.4262208 ,  1.3844621 , -0.10485933,  0.8241641 , -0.30078936,
       -0.5806731 ,  0.02172874, -0.1760632 ,  0.15458111,  0.6214116 ,
        0.2582276 , -0.4465092 ,  0.48415485,  0.8548936 , -0.11

In [21]:
black_eyed_peas_vector

array([ 0.00414648,  0.0291456 ,  0.01146119,  0.01609373,  0.0126302 ,
        0.01129968, -0.0080808 ,  0.01623365,  0.00536855, -0.00083252,
        0.00600557,  0.00978122,  0.02642579,  0.01564296, -0.00284582,
        0.00191864, -0.00683049,  0.00591398, -0.00205135,  0.0131644 ,
        0.00379372,  0.00284219,  0.00225235,  0.01065941, -0.00635788,
       -0.00344026, -0.0014126 ,  0.00396926,  0.02319785,  0.0256491 ,
        0.0252398 ,  0.01681733,  0.01968692,  0.00153903, -0.00354605,
       -0.01760771,  0.0046314 ,  0.01521238, -0.00788262,  0.00841332,
       -0.01651036, -0.0020454 ,  0.00978088,  0.02401309,  0.00495964,
        0.00349433,  0.01568057, -0.00278972,  0.00134984, -0.00512584,
       -0.00191644,  0.00778993,  0.01201452,  0.01640328,  0.00381944,
       -0.0121887 ,  0.02985895,  0.0218248 ,  0.0219588 ,  0.0019173 ,
       -0.01753967,  0.00404958,  0.01217789,  0.01545108,  0.02922773,
        0.00116905,  0.01671151,  0.01219289,  0.02836092,  0.00

In [22]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.5103259

In [23]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.31623173

### 비슷한 아티스트 찾기 + 유저에게 추천하기

In [24]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.12569489),
 (277, 0.1236978),
 (28, 0.12294108),
 (5, 0.12285188),
 (217, 0.122234486),
 (473, 0.12178948),
 (247, 0.120660916),
 (490, 0.12061905),
 (694, 0.1203242),
 (418, 0.11977204),
 (1018, 0.11916965),
 (910, 0.118911386),
 (268, 0.1184265),
 (55, 0.1176449),
 (782, 0.11711306)]

In [25]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'placebo',
 'the beatles',
 'oasis',
 'foo fighters',
 'u2',
 'the smashing pumpkins',
 'nirvana',
 'pink floyd',
 'arctic monkeys',
 'the white stripes']

In [26]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

In [27]:
get_similar_artist('2pac')

['2pac',
 'nas',
 'notorious b.i.g.',
 '50 cent',
 'jay-z',
 'dr. dre',
 'the game',
 'snoop dogg',
 'ludacris',
 'common']

In [28]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'the pussycat dolls',
 'beyoncé',
 'christina aguilera',
 'kelly clarkson',
 'justin timberlake',
 'leona lewis']

In [29]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.47056693),
 (550, 0.46015283),
 (627, 0.43948534),
 (369, 0.43525627),
 (24, 0.43511653),
 (1800, 0.4319268),
 (274, 0.43110004),
 (354, 0.41860375),
 (2249, 0.40604645),
 (391, 0.4025916),
 (409, 0.39708245),
 (382, 0.39125493),
 (618, 0.38975245),
 (5555, 0.38824934),
 (621, 0.3854998),
 (724, 0.38459295),
 (355, 0.38166946),
 (901, 0.3748519),
 (1777, 0.37043935),
 (564, 0.3655578)]

In [30]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'britney spears',
 'maroon 5',
 'justin timberlake',
 'jack johnson',
 'lady gaga',
 'michael jackson',
 'nelly furtado',
 'katy perry',
 'christina aguilera',
 'amy winehouse',
 'mika',
 'the pussycat dolls',
 'timbaland',
 'alicia keys',
 'lily allen',
 'madonna',
 'pink',
 'mariah carey',
 'kanye west']

In [33]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.23093431564564507),
 ('black eyed peas', 0.14061180915298593),
 ('jason mraz', 0.05647996556802095),
 ('coldplay', 0.04574007859368802),
 ('maroon5', -0.000136873145766962)]